# 最終課題

Kaggle Username: ysmith1hitu

In [1]:
import pandas as pd
import seaborn as sns

In [5]:
df = pd.read_csv("ainyuumon2021winter/train.csv", index_col=0)
df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
ID,,,,,,,,,
3401,Bachelors,2018,Bangalore,3,41,Male,No,4,1
1463,Bachelors,2012,Bangalore,3,27,Male,No,5,0
4644,Bachelors,2013,Bangalore,3,31,Female,No,5,0
3303,Bachelors,2016,Bangalore,1,39,Male,No,2,0
1260,Bachelors,2014,Pune,3,26,Male,No,4,0
...,...,...,...,...,...,...,...,...,...
1149,Bachelors,2013,New Delhi,2,25,Female,No,3,1
3579,Masters,2015,New Delhi,3,39,Female,No,2,0
3600,Bachelors,2015,Pune,2,32,Female,No,3,1


In [40]:
def preprocess(_df):
    x = _df[["PaymentTier", "Age", "ExperienceInCurrentDomain"]]
    education = pd.get_dummies(_df["Education"], drop_first=True, prefix="Education")
    city = pd.get_dummies(_df["City"], drop_first=True, prefix="City")
    gender = pd.get_dummies(_df["Gender"], drop_first=True, prefix="Gender")
    elapsed = (_df["JoiningYear"] - _df["JoiningYear"].max()) * -1
    elapsed.name = "ElapsedYears"
    x = pd.concat([x, education, city, gender, elapsed], axis=1)
    return x

In [41]:
Y = df["LeaveOrNot"]
X = preprocess(df)
X

,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,ElapsedYears
ID,,,,,,,,,
3401,3,41,4,0,0,0,0,1,0
1463,3,27,5,0,0,0,0,1,6
4644,3,31,5,0,0,0,0,0,5
3303,1,39,2,0,0,0,0,1,2
1260,3,26,4,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
1149,2,25,3,0,0,1,0,0,5
3579,3,39,2,1,0,1,0,0,3
3600,2,32,3,0,0,0,1,0,3


In [43]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.svm import SVR

In [59]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

trans = PowerTransformer()
trans.fit(X_train)

# gs = GridSearchCV(
#     estimator=SVR(),
#     param_grid={
#         "C": [0.001, 0.01, 0.1, 1, 10, 100],
#         "gamma": [0.001, 0.01, 0.1, 1, 10, 100],
#     },
#     cv=5
# )
# gs.fit(trans.transform(X_train), Y_train)
#
# print(gs.best_score_)
# print(gs.best_params_)
# print(gs.score(trans.transform(X_test), Y_test))

model = LogisticRegression()
model.fit(trans.transform(X_train), Y_train)

print(f"train: {model.score(trans.transform(X_train), Y_train)}")
print(f"test: {model.score(trans.transform(X_test), Y_test)}")

0.40206149138978614
{'C': 1, 'gamma': 0.1}
0.4218568180487645
train: 0.7395238095238095
test: 0.7585714285714286


In [52]:
df_test = pd.read_csv("ainyuumon2021winter/test.csv")
df_test

,ID,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,3189,Bachelors,2015,New Delhi,3,40,Male,No,2
1,4016,PHD,2018,New Delhi,3,34,Male,No,0
2,2830,Bachelors,2014,Bangalore,3,27,Male,Yes,5
3,2239,Bachelors,2013,New Delhi,3,27,Male,No,5
4,1047,Bachelors,2012,Bangalore,3,28,Male,No,1
...,...,...,...,...,...,...,...,...,...
1848,2241,Masters,2017,New Delhi,2,28,Male,No,1
1849,2769,Masters,2013,New Delhi,2,26,Male,No,4
1850,3753,Masters,2017,Pune,2,39,Male,No,2
1851,4233,Bachelors,2015,New Delhi,3,33,Female,No,5


In [53]:
X_test = preprocess(df_test)
X_test

,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,ElapsedYears
0,3,40,2,0,0,1,0,1,3
1,3,34,0,0,1,1,0,1,0
2,3,27,5,0,0,0,0,1,4
3,3,27,5,0,0,1,0,1,5
4,3,28,1,0,0,0,0,1,6
...,...,...,...,...,...,...,...,...,...
1848,2,28,1,1,0,1,0,1,1
1849,2,26,4,1,0,1,0,1,5
1850,2,39,2,1,0,0,1,1,1
1851,3,33,5,0,0,1,0,0,3


In [55]:
Y_pred = model.predict(trans.transform(X_test))

In [56]:
submission = pd.concat([df_test["ID"], pd.DataFrame(Y_pred, columns=["Pred"])], axis=1)
submission

,ID,Pred
0,3189,0
1,4016,0
2,2830,0
3,2239,0
4,1047,0
...,...,...
1848,2241,0
1849,2769,0
1850,3753,1
1851,4233,0


In [58]:
submission.to_csv("submission.csv", index=False)